In [3]:
from openbabel import openbabel
from openbabel import pybel
import openbabel
import sys
import pymatgen
import rdkit
from rdkit import Chem
import tensorflow as tf
from megnet.models import MEGNetModel
from megnet.models import GraphModel
from tensorflow.keras import optimizers
from megnet.data.molecule import MolecularGraph
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import r2_score
nfeat_bond = 100
r_cutoff = 10
gaussian_centers = np.linspace(0, r_cutoff + 1, nfeat_bond)
gaussian_width = 0.5


#BUILD MODEL

smiles = np.loadtxt('smiles.csv', dtype=str,delimiter = ',')
state_attributes = np.loadtxt('Reaxys_n.csv', delimiter = ',')
Dy_bin = np.loadtxt('y_bin.csv',delimiter = ',')
cutoff = 0.7
                
# #gives a molecule structure
structures =[]
failed_smi = []
for smi in smiles:
    try:
        structures.append(pybel.readstring("smi",smi))
    except:
        failed_smi.append(list(np.ravel(smiles)).index(smi))
print(np.shape(state_attributes))
Dy_bin = Dy_bin.astype('float32')
Dy_bin = [j for i,j in enumerate(Dy_bin) if i not in failed_smi]
state_attributes = np.delete(state_attributes,failed_smi,axis=0)



(161, 13)


In [4]:
tf.keras.backend.clear_session()

graphs_valid = []
targets_valid = []
structures_invalid = []
del_materials =[]
x=0


model = MEGNetModel(graph_converter = MolecularGraph(),centers = gaussian_centers,width = gaussian_width,
                   nfeat_node=27,nfeat_edge=27,nfeat_global=len(state_attributes[0]))

for s, p in zip(structures, Dy_bin):
    try:
        graph = model.graph_converter.convert(s,state_attributes = [list(state_attributes[x])])
        if np.any(np.isinf(graph['bond']))==False and np.any(np.isinf(graph['atom']))==False and np.any(np.isinf(graph['state']))==False:
            for i in range(len(graph['atom'])):
                graph['atom'][i] = [float(y) for y in graph['atom'][i]]
            graphs_valid.append(graph)

            targets_valid.append(p)
        else:
            graphs_valid.append(graph)
            targets_valid.append(p)
    except:
        structures_invalid.append(s)
        del_materials.append(x)
    x+=1

Dy_bin = [j for i,j in enumerate(Dy_bin) if i not in del_materials]

graphs_valid = np.array(graphs_valid)
targets_valid_split = np.array(Dy_bin)
targets_valid = tf.keras.utils.to_categorical(np.array(Dy_bin))

skf = StratifiedKFold(n_splits = 5)
predicted_y = []
for train_index, test_index in skf.split(graphs_valid,targets_valid_split):
    pots = int(len(train_index)*.8)
    
    graphs = np.array(graphs_valid[train_index])
    targets = np.array(targets_valid[train_index])
    
    graph_train = graphs[:pots]
    graph_validation = graphs[pots:]
    target_train = tf.keras.utils.to_categorical(targets[:pots])
    target_validation = tf.keras.utils.to_categorical(targets[pots:])
    
    model.train_from_graphs(train_graphs=graph_train, train_targets = target_train, 
                       validation_graphs=graph_validation,validation_targets=target_validation)#,
                            #epochs=100, verbose=0,batch_size=100)
    predicted_y.extend([model.predict_graph(x) for x in graphs[test_index]])

fig, ax = plt.subplots()

ax.scatter(targets_valid, predicted_y)

# Add the y=x line
lim = [min(ax.get_xlim() + ax.get_ylim()), max(ax.get_xlim() + ax.get_ylim())]
ax.set_xlim((-0.05,1.05))
ax.set_ylim((-0.05,1.05))
ax.plot(lim, lim, 'k--')

ax.set_xlabel('baseline')
ax.set_ylabel('predicted')

fig.set_size_inches(5,5)
fig.tight_layout()

fig.show()

plt.savefig(Label+'.png')
predicted_y_scaled = [1 if i>cutoff else 0 for i in predicted_y]
targets_valid_scaled = [1 if i>cutoff else 0 for i in targets_valid]
cm = confusion_matrix(targets_alid_scaled, predicted_y_scaled)
TP,TN,FP,FN = cm[1,1],cm[0,0],cm[0,1],cm[1,0]

comp = sum([1 for i in list(range(len(predicted_y))) if predicted_y_scaled[i] == targets_alid_scaled[i]])
accuracy = comp/len(pred_y)

print('Accuracy = ',str(accuracy),' Precision = ',str(TP/(TP+FP)),' Recall = ',str(TP/(TP+FN)))

Epoch 1/1000


/home/bransom/Programs/anaconda3/envs/tf-8/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_atom/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_atom/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_atom/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/home/bransom/Programs/anaconda3/envs/tf-8/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_bond/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_bond/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_bond/Cast:0", shape=(1,), dtype=int

1/1 [==============================] - 15s 15s/step - loss: nan


INFO:megnet.callbacks:Nan loss found!
INFO:megnet.callbacks:Load weights callback/val_mae_00882_0.009824.hdf5
INFO:megnet.callbacks:Now lr is 0.0005000000237487257.


Epoch 2/1000


TypeError: 'NoneType' object is not callable